# read tracks

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('./tracks_result.csv')

In [8]:
df_gp = df.groupby('frame')
first_gp_name = next(iter(df_gp.groups))
first_gp = df_gp.get_group(first_gp_name)

print(first_gp)

      frame  trackId            timestamp  vehicle_id     xCenter    yCenter  \
0         0        1  1706928726990468352         1.0    1.525176  -0.081205   
998       0        2  1706928726990468352    354695.0  -95.356619   4.459940   
1405      0        3  1706928726990468352    386711.0   24.248349  -2.244065   
2400      0        4  1706928726990468352    390316.0  -49.688790  -0.040203   
3224      0        5  1706928726990468352    395774.0   46.495755   1.270597   
4219      0        6  1706928726990468352    396827.0   39.495966  -1.225302   
5214      0        7  1706928726990468352    398015.0  -22.697545  -3.306698   
6209      0        8  1706928726990468352    402579.0  129.294221   0.261016   
7204      0        9  1706928726990468352    403803.0   11.006345 -12.296713   
7238      0       10  1706928726990468352    403922.0  -69.770702  -2.928023   

        length     width    height  xVelocity  ...  rightPrecedingId  \
0     3.000000  1.980000  1.680000  15.805735  

In [9]:
RU = ["Car", "SUV", "Truck", "Special_truck"]
VRU = ["tricycle", "bicycle", "motorbike"]

first_gp.columns

Index(['frame', 'trackId', 'timestamp', 'vehicle_id', 'xCenter', 'yCenter',
       'length', 'width', 'height', 'xVelocity', 'yVelocity', 'xAcceleration',
       'yAcceleration', 'frontSightDistance', 'backSightDistance', 'dhw',
       'thw', 'ttc', 'precedingXVelocity', 'precedingId', 'followingId',
       'leftPrecedingId', 'leftAlongsideId', 'leftFollowingId',
       'rightPrecedingId', 'rightAlongsideId', 'rightFollowingId', 'roadId',
       'laneId', 'angle', 'orientation', 'yaw', 'yaw_rate', 'ego_offset'],
      dtype='object')

In [10]:
for gp_frame, df_gp in df.groupby("frame"):
    # each frame, we have a list of RUs
    
    # for each RU
    for ru in df_gp:
        # each RU in this frame
        pass

SyntaxError: expected ':' (502002093.py, line 1)

# Lane Sorting

In [1]:

def label_vehicles_by_lane_with_ego_at_zero(df, ego_id, xvel, yvel, num_lanes=5, ego_lane_idx=2):
  """
  Labels vehicles on different lanes from left to right, 
  sorts vehicles within each lane from front to back, 
  and distinguishes between front and behind the ego vehicle.
  Ego vehicle is always at index 0 within its lane.

  Args:
    df: pandas DataFrame containing vehicle data with columns:
      - 'x': x-coordinate of the vehicle
      - 'y': y-coordinate of the vehicle
      - 'lane_id': lane ID of the vehicle
      - 'id': unique identifier for each vehicle
    ego_id: unique identifier of the ego vehicle
    xvel: x-component of ego's velocity
    yvel: y-component of ego's velocity
    num_lanes: Total number of lanes (default: 5)
    ego_lane_idx: Index of the ego vehicle's lane (0-indexed, default: 2)

  Returns:
    pandas DataFrame with an additional column:
      - 'relative_position': 
          - 'front' if the vehicle is in front of the ego vehicle
          - 'behind' if the vehicle is behind the ego vehicle
          - 'ego' if the vehicle is the ego vehicle
          - 'other_lane' if the vehicle is on a different lane
  """

  df['relative_position'] = 'other_lane'

  # Calculate lane direction vector
  lane_direction = np.array([xvel, yvel])

  # Calculate relative positions to ego vehicle
  ego_vehicle = df[df['id'] == ego_id]
  if ego_vehicle.empty:
    raise ValueError("Ego vehicle not found in the DataFrame.")
  ego_position = ego_vehicle[['x', 'y']].values[0]
  df['rel_x'] = df['x'] - ego_position[0]
  df['rel_y'] = df['y'] - ego_position[1]

  # Project relative positions onto lane direction
  df['projection'] = np.dot(df[['rel_x', 'rel_y']].values, lane_direction)

  # Group vehicles by lane
  for i in range(num_lanes): 
    lane_vehicles = df[df['lane_id'] == i]
    if lane_vehicles.empty:
      continue

    # Sort vehicles within each lane
    sorted_lane_vehicles = lane_vehicles.sort_values(by='projection')

    # Reindex to put ego vehicle at index 0
    ego_index = sorted_lane_vehicles.index[sorted_lane_vehicles['id'] == ego_id].tolist()
    if ego_index:
      ego_index = ego_index[0]
      sorted_lane_vehicles = pd.concat([sorted_lane_vehicles.iloc[ego_index:], sorted_lane_vehicles.iloc[:ego_index]])

    # Label vehicles relative to the ego vehicle
    sorted_lane_vehicles.loc[0, 'relative_position'] = 'ego'
    sorted_lane_vehicles.loc[1:, 'relative_position'] = 'front'
    sorted_lane_vehicles.loc[:0, 'relative_position'] = 'behind' 

    # Update the original DataFrame
    df.loc[lane_vehicles.index, 'relative_position'] = sorted_lane_vehicles['relative_position']

  return df

# Example usage
data = {'id': ['v1', 'v2', 'ego', 'v4', 'v5', 'v6', 'v7'],
        'x': [10, 20, 30, 40, 50, 15, 25],
        'y': [1, 1, 1, 1, 1, 2, 2], 
        'lane_id': [0, 0, 0, 1, 1, 2, 2]}  # Assuming 3 lanes (0, 1, 2)
df = pd.DataFrame(data)

labeled_df = label_vehicles_by_lane_with_ego_at_zero(df, 'ego', 1, 0, num_lanes=3, ego_lane_idx=0) 
print(labeled_df)

KeyError: 1